In [1]:
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import show
output_notebook()

Loading BokehJS ...

This is the dataset. The *shape* is required. In this case it is WKT encoded, but it could also be [GeoJSON](https://geojson.org/) strings, [Geo-Interface](https://sgillies.net/2013/06/27/geojson-and-the-geo-interface-for-python.html) dicts, or [Shapely](https://shapely.readthedocs.io/en/stable/manual.html) geometries.
The other columns are optional. *color* is ... well ... the color, and *alpha* is the opacity. The last two, *date* and *classification*, will be turned into tooltips.

In [2]:
bigfoot_sightings = pd.read_csv("../data/bigfoot_sightings.csv")
bigfoot_sightings.head()

,shape,color,alpha,date,classification
0,POINT (-95.5425 32.7943),black,1.0,1996-12-22,Class A
1,POINT (-95.61056000000001 32.69028),black,1.0,2004-03-25,Class A
2,POINT (-95.46671000000001 32.85232),black,1.0,2006-02-01,Class A
3,POINT (-97.83750000000001 33.29083),black,1.0,2003-10-10,Class A
4,POINT (-99.295 34.31),black,1.0,2016-07-23,Class B


The main function in map maker, `make_map_plot`, takes a list of dictionaries. Transforming the data frame to this is simple - just use `.as_dict("records")`.

In [3]:
bigfoot_sightings.to_dict("records")[:2]

[{'shape': 'POINT (-95.5425 32.7943)',
  'color': 'black',
  'alpha': 1.0,
  'date': '1996-12-22',
  'classification': 'Class A'},
 {'shape': 'POINT (-95.61056000000001 32.69028)',
  'color': 'black',
  'alpha': 1.0,
  'date': '2004-03-25',
  'classification': 'Class A'}]

`make_map_plot` (in the `bokeh` module) returns a Bokeh [Figure](https://docs.bokeh.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure) object. Just use `show()` to display it like you would any Bokeh plot.

In [4]:
from map_maker.bokeh import make_map_plot
bigfoot_sightings_map = make_map_plot(
    bigfoot_sightings.to_dict("records"),
    plot_height=500,
    plot_width=600
)
show(bigfoot_sightings_map)

The dicts in the list for `make_map_plot` do not need to be homogenous. The extra columns (or color / alpha) can be different for different tooltips. And map maker doesn't care if the geometries are different either. Let's add another dataset.

In [5]:
texas_cities = pd.read_csv("../data/texas_cities.csv")
texas_cities.head()

,shape,color,alpha,city,size
0,POLYGON ((-97.22350292727579 30.26616919987027...,lightblue,0.2,Austin,2.5
1,POLYGON ((-97.97834927165552 29.42310357869992...,lightblue,0.2,San Antonio,2.5
2,POLYGON ((-96.26332323120107 32.77556329501405...,lightblue,0.2,Dallas,2.5
3,POLYGON ((-94.85283773661263 29.75938993355712...,lightblue,0.2,Houston,2.5
4,"LINESTRING (-97.7431 30.2672, -96.797 32.7767)",green,1.0,Austin-Dallas,3.0


This dataset contains several major cities in Texas with buffers drawn around them as semi-transparent polygons, as well as a few lines connecting them. Adding them to the bigfoot sightings is as simple as converting both to a list of dicts and then concatenating them.

While we're at it, let's increase the point size too.

In [6]:
bigfoot_sightings_map = make_map_plot(
    bigfoot_sightings.to_dict("records") + texas_cities.to_dict("records"),
    # By default only points have tooltips enabled. Use tooltips to turn them
    # on for linestrings and polygons as well.
    tooltips={"points", "linestrings", "polygons"},
    # point_size sets ... the point size. The units are passed directly to
    # bokeh. You can also set the linestring width with linestring_width and
    # the line width of the polygons with polygon_line_width.
    point_size=5,
    plot_height=500,
    plot_width=600
)
show(bigfoot_sightings_map)

If your map needs seem a little _too_ complicated - complex interactions and whatnot - you can always go one level deeper and use a lower level library. You can also use `make_map_plot`'s output as a starting point. It's nothing fancy - just a bokeh `Figure` object. Any of the other bokeh stuff applies here too.